In [ ]:
import numpy as np
import rasterio as rio
from rasterio.plot import show
import matplotlib.pyplot as plt
from pathlib import Path
import ee

# Initialize Earth Engine
try:
    ee.Initialize(project='ee-geos505-ubrb-prep')
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='ee-geos505-ubrb-prep')
    
aso_path = '../data/aso_data/'

gdrive_path = '_coge_geos505'
gdrive_path_local = f'/Users/lejoflores/Google Drive/My Drive/{gdrive_path}/'

coge_fname = 'coge_dem_50m'

directory_path = Path(aso_path)

In [ ]:
aso_files = directory_path.glob('*.tif')
aso_files = sorted(aso_files)
print(aso_files)

In [ ]:
aso_data = rio.open(aso_files[0])
aso_array = aso_data.read(1)

aso_crs = str(aso_data.crs)
print(aso_crs)

In [ ]:
aso_array = np.where(aso_array == -9999, np.nan, aso_array)

In [ ]:
plt.rcParams.update({'font.size': 14})
fig, ax = plt.subplots(figsize=(14, 10))
show(aso_data, ax=ax, cmap='winter_r', alpha=0.5)
ax.set_xlabel('Easting (m)')
ax.set_ylabel('Northing (m)')
plt.show()

In [ ]:
# Create an ee.Geometry.Rectangle with the specified UTM coordinates and projection
aso_bounding_box = ee.Geometry.Rectangle([aso_data.bounds.left, 
                                          aso_data.bounds.bottom, 
                                          aso_data.bounds.right - aso_data.transform.a,
                                          aso_data.bounds.top + aso_data.transform.e], aso_crs, False)


In [ ]:
# Import the USGS NED dataset (30m DEM)
dem_collection = ee.ImageCollection('USGS/3DEP/10m_collection')

dem_image = dem_collection.mosaic().select('elevation')

dem_image_utm = dem_image.reproject(crs=aso_crs, scale=aso_data.transform.a)

In [ ]:
# Clip the DEM to the Gunnison-East headwater basin
aso_clipped_dem = dem_image_utm.clip(aso_bounding_box)

In [ ]:
# Aggregate the DEM to a 30m spatial resolution using mean aggregation.
# Note that this will require resampling which may introduce some error.
aso_aggregated_dem = aso_clipped_dem.reduceResolution(
    reducer=ee.Reducer.mean(),
    maxPixels=1024
).reproject(
    crs=aso_clipped_dem.projection().crs(),
    scale=aso_data.transform.a
)


In [ ]:
# Export the image to Google Drive
task_config = {
    'folder': gdrive_path,  # Replace with desired folder name
    'region': aso_bounding_box,
    'scale': aso_data.transform.a,
    'fileFormat': 'GeoTIFF',
    'fileNamePrefix': coge_fname,
    'maxPixels': 1e13  # Adjust as needed based on image size
}

task = ee.batch.Export.image.toDrive(aso_aggregated_dem.select('elevation'), **task_config)
task.start()

print('Export task started. Check your Google Drive for the results.')


In [ ]:
task.status()

In [ ]:
elev_data = rio.open(gdrive_path_local + coge_fname + '.tif')
elev_array = elev_data.read(1)
elev_array.shape

In [ ]:
aso_array.shape

In [ ]:
plt.rcParams.update({'font.size': 14})
fig, ax = plt.subplots(figsize=(14, 10))
show(elev_data, ax=ax, cmap='bone')
show(aso_data, ax=ax, cmap='Reds', alpha=0.5)
ax.set_xlabel('Easting (m)')
ax.set_ylabel('Northing (m)')
plt.show()
